<a href="https://colab.research.google.com/github/RashmiSunkara/Landslide-Identification/blob/main/HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hmmlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.2/217.2 KB 7.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from hmmlearn import hmm

Data Exploration and Preprocessing

In [ ]:
#loading data
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Major Project/Global_Landslide_Final.csv")

In [ ]:
data.head()

,landslide_trigger,event_id,formatted_date,event_date,event_month,event_day,event_time,event_title,event_description,location_description,...,landslide_size,fatality_count,injury_count,country_name,country_code,population,gazeteer_closest_point,gazeteer_distance,longitude,latitude
0,Construction,7544,7/14/2015 23:00,7/14/2015,7,14,23:00,Highbury Rd and Huntingdale Rd,WITNESSES have detailed the dramatic moment th...,Highbury Rd and Huntingdale Rd,...,small,NaN,NaN,Australia,AU,11886.0,Burwood,0.77420,145.1147,-37.8558
1,Construction,7616,08-06-2015 00:00,08-06-2015,8,6,09:00,Atlanta Point,"The PBMC Chairperson, Mr. Armugam has made an ...",Atlanta Point,...,medium,0.0,0.0,India,IN,112050.0,Port Blair,0.96496,92.7490,11.6753
2,Construction,9558,9/27/2016 0:00,9/27/2016,9,27,00:00,Basaksehir district,People had been working both at the top and at...,Basaksehir district,...,medium,0.0,0.0,Turkey,TR,193750.0,BaÅŸakÅŸehir,2.78453,28.7701,41.0863
3,Construction,6480,08-12-2014 00:00,08-12-2014,8,12,02:30,the metro bus flyover project in Sector G9,The labourers were busy in work on the metro b...,the metro bus flyover project in Sector G9,...,small,2.0,4.0,Pakistan,PK,601600.0,Islamabad,3.05654,73.0399,33.6941
4,Construction,6030,5/17/2014 0:00,5/17/2014,5,17,09:00,Dariali Gorge,A landslide in the Dariali Gorge in Georgia ha...,Dariali Gorge,...,medium,8.0,0.0,Georgia,GE,2628.0,Stepâ€™antsminda,9.57790,44.5015,42.7204


In [ ]:
data.dtypes

landslide_trigger          object
event_id                    int64
formatted_date             object
event_date                 object
event_month                 int64
event_day                   int64
event_time                 object
event_title                object
event_description          object
location_description       object
location_accuracy          object
landslide_category         object
landslide_size             object
fatality_count            float64
injury_count              float64
country_name               object
country_code               object
population                float64
gazeteer_closest_point     object
gazeteer_distance         float64
longitude                 float64
latitude                  float64
dtype: object

In [ ]:
data.isna().sum()

landslide_trigger            0
event_id                     0
formatted_date               0
event_date                   0
event_month                  0
event_day                    0
event_time                 397
event_title                  0
event_description           86
location_description        81
location_accuracy            1
landslide_category           0
landslide_size               0
fatality_count            1321
injury_count              5458
country_name              1160
country_code              1314
population                1312
gazeteer_closest_point    1313
gazeteer_distance         1312
longitude                    0
latitude                     0
dtype: int64

Observed States : landslide_trigger,landslide_size,gazeteer_closest_point,gazeteer_distance

Hidden States : landslide_category which has 9 states namely slide,complex,rock_fall,earth_fall,debris_flow and snow_avalanche

In [ ]:
# Get the unique values in a column
states =data['landslide_size'].unique()

# Print the unique states
print(states)

['small' 'medium' 'large' 'unknown' 'very_large' 'catastrophic']


In [ ]:
# Replace 'unknown' values with NaN
data['landslide_size'] = data['landslide_size'].replace('unknown', np.nan)
landslide_size_mode=data['landslide_size'].mode()[0]
# Fill NaN values with the mode
data['landslide_size'] = data['landslide_size'].fillna(landslide_size_mode)


In [ ]:
states =data['landslide_size'].unique()
print(states)

['small' 'medium' 'large' 'very_large' 'catastrophic']


In [ ]:
# Map the landslide_size column to integer values
size_map = {'small': 0, 'medium': 1, 'large': 2, 'very_large': 3, 'catastrophic' : 4}
data['landslide_size_int'] = data['landslide_size'].map(size_map)
# Remove the original landslide_size column
data = data.drop('landslide_size', axis=1)

In [ ]:
pos = [ sum(y>=0 for y in data['landslide_size_int']) ]
neg = [ len(data['landslide_size_int'])-x for x in pos]
print(pos, neg)

[9132] [0]


In [ ]:
states =data['landslide_trigger'].unique()
print(states)

['Construction' 'Earthquake' 'Flooding' 'Mining' 'Monsoon' 'Rain' 'Snow'
 'Tropical_Cyclone']


In [ ]:
# Map the landslide_size column to integer values
size_map2 = {'Construction': 0, 'Earthquake': 1, 'Flooding': 2, 'Mining': 3, 'Monsoon' : 4, 'Rain' : 5, 'Snow' : 6, 'Tropical_Cyclone' : 7}
data['landslide_trigger_int'] = data['landslide_trigger'].map(size_map2)
# Remove the original landslide_size column
data = data.drop('landslide_trigger', axis=1)

In [ ]:
pos = [ sum(y>=0 for y in data['landslide_trigger_int']) ]
neg = [ len(data['landslide_trigger_int'])-x for x in pos]
print(pos, neg)

[9132] [0]


In [ ]:
data.head()

,event_id,formatted_date,event_date,event_month,event_day,event_time,event_title,event_description,location_description,location_accuracy,...,injury_count,country_name,country_code,population,gazeteer_closest_point,gazeteer_distance,longitude,latitude,landslide_size_int,landslide_trigger_int
0,7544,7/14/2015 23:00,7/14/2015,7,14,23:00,Highbury Rd and Huntingdale Rd,WITNESSES have detailed the dramatic moment th...,Highbury Rd and Huntingdale Rd,1km,...,NaN,Australia,AU,11886.0,Burwood,0.77420,145.1147,-37.8558,0,0
1,7616,08-06-2015 00:00,08-06-2015,8,6,09:00,Atlanta Point,"The PBMC Chairperson, Mr. Armugam has made an ...",Atlanta Point,1km,...,0.0,India,IN,112050.0,Port Blair,0.96496,92.7490,11.6753,1,0
2,9558,9/27/2016 0:00,9/27/2016,9,27,00:00,Basaksehir district,People had been working both at the top and at...,Basaksehir district,10km,...,0.0,Turkey,TR,193750.0,BaÅŸakÅŸehir,2.78453,28.7701,41.0863,1,0
3,6480,08-12-2014 00:00,08-12-2014,8,12,02:30,the metro bus flyover project in Sector G9,The labourers were busy in work on the metro b...,the metro bus flyover project in Sector G9,25km,...,4.0,Pakistan,PK,601600.0,Islamabad,3.05654,73.0399,33.6941,0,0
4,6030,5/17/2014 0:00,5/17/2014,5,17,09:00,Dariali Gorge,A landslide in the Dariali Gorge in Georgia ha...,Dariali Gorge,10km,...,0.0,Georgia,GE,2628.0,Stepâ€™antsminda,9.57790,44.5015,42.7204,1,0


In [ ]:
observed_states=data[['landslide_size_int','landslide_trigger_int']].values


In [ ]:
states =data['landslide_category'].unique()
print(states)

['slide' 'rock_fall' 'complex' 'earth_fall' 'earth_flow' 'debris_flow'
 'snow_avalanche' 'lahar' 'creep']


In [ ]:
hidden_states=data['landslide_category'].values

In [ ]:
from sklearn.model_selection import train_test_split
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(observed_states, hidden_states, test_size=0.2, random_state=42)

In [ ]:
# Fit the model on the training set
model = hmm.CategoricalHMM(n_components=9)
model.fit(X_train)


CategoricalHMM(n_components=9,
               random_state=RandomState(MT19937) at 0x7FC825213340)

In [ ]:
ls=int(input("small : 0, medium : 1, large : 2, very_large : 3, catastrophic : 4 Enter Landslide size : "))
lt=int(input("Construction : 0, Earthquake : 1, Flooding : 2, Mining : 3, Monsoon : 4, Rain : 5, Snow : 6, Tropical_Cyclone : 7 Enter Landslide trigger : "))

In [ ]:

input_data = pd.DataFrame({'landslide_size_int': [ls], 'landslide_trigger_int': [lt]})
pred=model.predict(input_data)

In [ ]:
print(pred)

In [ ]:
# Define categorical states
categories = ['slide', 'rock_fall', 'complex', 'earth_fall', 'earth_flow', 'debris_flow', 'snow_avalanche', 'lahar', 'creep']
# Define a lookup table to map states to categories
state_to_category = {i: category for i, category in enumerate(categories)}
# Map the state sequence to the corresponding category
predicted_category = state_to_category[pred[0]]
predicted_category1 = state_to_category[pred[1]]

In [ ]:
# Get the probability distribution over all possible states for the given input
proba = model.predict_proba(input_data)
# Get the index of the predicted category
pred_category_idx = categories.index(predicted_category)
# Get the probability of the predicted category
pred_category_proba = proba[0][pred_category_idx]
pred_category_proba1 = proba[1][pred_category_idx]

In [ ]:
print(predicted_category)
print("Probability of predicted category :", pred_category_proba)

In [ ]:

print(predicted_category1)
print("Probability of predicted category:", pred_category_proba1)


In [ ]:
import landsatxplore.api
import landsatxplore.earthexplorer
import numpy as np
import rasterio.plot
import rasterio.warp
import matplotlib.pyplot as plt

# Define the latitude and longitude coordinates of the location
lat, lon = 40.7128, -74.0060

# Connect to the EarthExplorer API
ee = landsatxplore.earthexplorer.EarthExplorer('username', 'password')

# Search for Landsat 8 imagery that covers the given point
scenes = ee.search(
    dataset='LANDSAT_8_C1',
    latitude=lat,
    longitude=lon,
    start_date='2020-01-01',
    end_date='2020-12-31',
    max_cloud_cover=10,
    max_results=1
)

# Download the first scene in the search results
scene_id = scenes[0]['entityId']
ee.download(scene_id, output_dir='path/to/save/images')

# Open the downloaded image using rasterio
with rasterio.open('path/to/save/images/LC08_L1TP_012031_20200819_20200905_01_T1_B4.TIF') as src:
    # Reproject the image to Web Mercator for display in matplotlib
    transform, width, height = rasterio.warp.calculate_default_transform(
        src.crs, 'EPSG:3857', src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': 'EPSG:3857',
        'transform': transform,
        'width': width,
        'height': height
    })
    image = src.read(1, out_shape=(src.height, src.width))

# Display the image using matplotlib
plt.imshow(image, cmap='gray')
plt.show()
